<a href="https://colab.research.google.com/github/bmanikan/FallDetection/blob/main/Code%20notebooks/FallDetectionWandB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Note: This Notebook implements the algorithms and architectures for the falldetection problem using tensorflow and is similar to FallDetection_TF.ipynb. The only difference is that the log file is now documented in Weights and Biases website (wandb.com) for interactive visualization.**

## Logger

In [ ]:
import datetime
import time
import logging
import sys

def set_logger():
  logging.basicConfig(level=logging.INFO)
  logger_t = logging.getLogger('withoutlevel')
  logger = logging.getLogger('withlevel')

  f_handler = logging.FileHandler('Falldetection.log')
  f_handler_t = logging.FileHandler('Falldetection.log')

  f_format = logging.Formatter('%(levelname)s - %(message)s')
  f_format_t = logging.Formatter('%(message)s')

  f_handler.setFormatter(f_format)
  f_handler_t.setFormatter(f_format_t)

  logger.addHandler(f_handler)
  logger_t.addHandler(f_handler_t)
  return logger, logger_t

In [ ]:
import datetime
import time
t_set = lambda: datetime.datetime.now().astimezone()
t_diff = lambda t: str(t_set() - t)
w_diff = lambda t: (t_set() - t).total_seconds()
t_stamp = lambda t=None: str(t) if t else str(t_set())
t = t_set()
time.sleep(5)
t_diff(t)

'0:00:05.005365'

In [ ]:
import logging
import sys

#logger without time
logging.basicConfig(level=logging.INFO)
logger_t = logging.getLogger('withoutlevel')
logger = logging.getLogger('withlevel')

f_handler = logging.FileHandler('Falldetection.log')
f_handler_t = logging.FileHandler('Falldetection.log')

f_format = logging.Formatter('%(levelname)s - %(message)s')
f_format_t = logging.Formatter('%(message)s')

f_handler.setFormatter(f_format)
f_handler_t.setFormatter(f_format_t)

logger.addHandler(f_handler)
logger_t.addHandler(f_handler_t)


In [ ]:
import platform
platform.uname()
for det in platform.uname():
  logger_t.info(str(det))

INFO:withoutlevel:Linux
INFO:withoutlevel:11f15fe0b98a
INFO:withoutlevel:5.4.104+
INFO:withoutlevel:#1 SMP Sat Jun 5 09:50:34 PDT 2021
INFO:withoutlevel:x86_64
INFO:withoutlevel:x86_64


In [ ]:
logger_t.info('='*100)
logger_t.info("\nFalldetection project")

INFO:withoutlevel:====================================================================================================
INFO:withoutlevel:
Falldetection project


## config

In [ ]:
%%capture
!pip install wandb

In [ ]:
import tensorflow as tf
import numpy as np
import random
import os

#Set seed value
seed_value = 43

os.environ['PYTHONHASHSEED'] = str(seed_value)

random.seed(seed_value)

#numpy seed
np.random.seed(seed_value)

#Tf seed
tf.random.set_seed(seed_value)

#Configure new global tensorflow session
from tensorflow.compat.v1.keras import backend as k
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads = 1,
    inter_op_parallelism_threads = 1
)

sess = tf.compat.v1.Session(graph = tf.compat.v1.get_default_graph(), config = session_conf)
k.set_session(sess)

In [ ]:
import wandb 
wandb.login()
wandb.init(project="fall_detection",
          config={
              'frameCount'    :50,
              'data_size'     :120,
              'dataset'       :'urfd',
              'epochs'        :15,
              'seed_value'    :43,
              'split'         :0.8,
              'batch_size'    :32,
              'activation'    :'relu',
              'optimizer'     :'adam',
              'loss_fn'       :'binarycrossentropy',
              'dropout_rate'  :0.5
          })
cfg = wandb.config

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: manikandan (use `wandb login --relogin` to force relogin)


In [ ]:
logger.info(f"The analysis with dataset {cfg.dataset} and frame count {cfg.frameCount} and size of the data is {cfg.data_size}.")

INFO:withlevel:The analysis with dataset urfd and frame count 50 and size of the data is 120.


## Download dataset

In [ ]:
import urllib.request as urlreq
from zipfile import ZipFile
import shutil
import os
from tqdm.notebook import tqdm
import socket
socket.setdefaulttimeout(20)

logger_t.info(f'{"="*100} \n\nSTEP 1: Download dataset into the system')

def safedirs(path):
  if not os.path.exists(path):
    os.makedirs(path)

def extract_zip(src, dest, map):
    '''
    Extracts the contents downloaded from the URL
    '''
    zip_ref = ZipFile(src,'r')
    for name in zip_ref.namelist():
      ext = name.split('.')[-1]
      if ext in map.keys():
        dest = os.path.join(dest,map[ext])
        break
    zip_ref.extractall(dest)
    logger_t.info(f'... into {dest}')
    zip_ref.close()

def cleanup(dataPath):
  '''
  clean the temp files
  '''
  for folder in os.listdir(dataPath):
    if folder == 'temp':
      cleanup_path = os.path.join(dataPath,'temp')
      shutil.rmtree(cleanup_path)
      break
  return

def download(url, root):
  '''
  downloads the file from the url
  and put it in desired folder
  '''
  
  map = {'csv':'csv', 'zip':'temp', 'png':'frame', 'jpg':'frame', 'mp4':'video', 'avi':'video'}
  ext = url.split('.')[-1] 
  # raise if the url is of not file type
  assert ext in map.keys(), "invalid url"
  name = url.split('/')[-1]
  filePath = os.path.join(root, map[ext])
  safedirs(filePath)
  filename = os.path.join(filePath,name)
  if os.path.exists(filename):
    return 
  logger_t.info(f"Downloading data files from URL:{url}")
  
  while True:
    try:
      urlreq.urlretrieve(url, filename)
      #if the file downloaded is zip file then extract original files
      if ext == 'zip':
        extract_zip(filename,root,map)
    except Exception as e:
      print(f'error {e} at {url}')
      continue
    break
    
  
    

def urfd(dataPath,num=40,clean=False):
  '''
  download the files from the URFD data url
  dataPath: destination path
  num: number of files to download
  clean: whether to clean the temp files
  '''
  #csv file url's
  csv = ['http://fenix.univ.rzeszow.pl/~mkepski/ds/data/urfall-cam0-adls.csv',
         'http://fenix.univ.rzeszow.pl/~mkepski/ds/data/urfall-cam0-falls.csv']
  #download csv files
  for f in csv:
    t = t_set()
    download(f,dataPath)
    logger.info(f'Elapsed time for downloading and extracting the file is {t_diff(t)}.\n')
    wandb.log({'download':{'time':w_diff(t)}})
  #download frame zip files from the project website
  for n in tqdm(range(1,num+1)):
    # Add zeros to adapt files in Website
    if n < 10:
      n = '0'+str(n)
    #only 31 Fall sequences available
    if int(n)<31:
      t = t_set()
      fall_url = f'http://fenix.univ.rzeszow.pl/~mkepski/ds/data/fall-{n}-cam0-rgb.zip'
      download(fall_url,datapath)
      logger.info(f'Elapsed time for downloading and extracting the file is {t_diff(t)}\n')
      wandb.log({'download':{'time':w_diff(t)}})
    t=t_set()
    adl_url = f'http://fenix.univ.rzeszow.pl/~mkepski/ds/data/adl-{n}-cam0-rgb.zip'
    download(adl_url,dataPath)
    logger.info(f'Elapsed time for downloading and extracting the file is {t_diff(t)}\n')
    wandb.log({'download':{'time':w_diff(t)}})
    
    if n == num-1 and clean:
      cleanup(dataPath)




root = os.getcwd()
datapath = root + '/dataset/urfd'
logger.info(f'Dateset is downloaded in path {datapath}')
t= t_set()
urfd(datapath) 
logger.info(f'Total time for downloading and extracting entire dataset is {t_diff(t)}\n {"="*200}')
wandb.summary['Download_TotalTime'] = w_diff(t)

INFO:withoutlevel:==================================================================================================== 

STEP 1: Download dataset into the system
INFO:withlevel:Dateset is downloaded in path /content/dataset/urfd
INFO:withoutlevel:Downloading data files from URL:http://fenix.univ.rzeszow.pl/~mkepski/ds/data/urfall-cam0-adls.csv
INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:01.120561.

INFO:withoutlevel:Downloading data files from URL:http://fenix.univ.rzeszow.pl/~mkepski/ds/data/urfall-cam0-falls.csv
INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:00.822965.



INFO:withoutlevel:Downloading data files from URL:http://fenix.univ.rzeszow.pl/~mkepski/ds/data/fall-01-cam0-rgb.zip
INFO:withoutlevel:... into /content/dataset/urfd/frame
INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:04.165348

INFO:withoutlevel:Downloading data files from URL:http://fenix.univ.rzeszow.pl/~mkepski/ds/data/adl-01-cam0-rgb.zip
INFO:withoutlevel:... into /content/dataset/urfd/frame
INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:04.974817

INFO:withoutlevel:Downloading data files from URL:http://fenix.univ.rzeszow.pl/~mkepski/ds/data/fall-02-cam0-rgb.zip
INFO:withoutlevel:... into /content/dataset/urfd/frame
INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:03.443290

INFO:withoutlevel:Downloading data files from URL:http://fenix.univ.rzeszow.pl/~mkepski/ds/data/adl-02-cam0-rgb.zip
INFO:withoutlevel:... into /content/dataset/urfd/frame
INFO:withlevel:Elapsed time for downloading and extra

error timed out at http://fenix.univ.rzeszow.pl/~mkepski/ds/data/adl-20-cam0-rgb.zip
error <urlopen error timed out> at http://fenix.univ.rzeszow.pl/~mkepski/ds/data/adl-20-cam0-rgb.zip
error <urlopen error timed out> at http://fenix.univ.rzeszow.pl/~mkepski/ds/data/adl-20-cam0-rgb.zip


INFO:withoutlevel:... into /content/dataset/urfd/frame
INFO:withlevel:Elapsed time for downloading and extracting the file is 0:01:18.836966

INFO:withoutlevel:Downloading data files from URL:http://fenix.univ.rzeszow.pl/~mkepski/ds/data/fall-21-cam0-rgb.zip
INFO:withoutlevel:... into /content/dataset/urfd/frame
INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:02.111349

INFO:withoutlevel:Downloading data files from URL:http://fenix.univ.rzeszow.pl/~mkepski/ds/data/adl-21-cam0-rgb.zip
INFO:withoutlevel:... into /content/dataset/urfd/frame
INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:08.091012

INFO:withoutlevel:Downloading data files from URL:http://fenix.univ.rzeszow.pl/~mkepski/ds/data/fall-22-cam0-rgb.zip
INFO:withoutlevel:... into /content/dataset/urfd/frame
INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:02.201704

INFO:withoutlevel:Downloading data files from URL:http://fenix.univ.rzeszow.pl/~mk

## Label enrichment


In [ ]:
import pandas as pd
import os

logger_t.info(f'{"="*200} \n\nSTEP 2: Assign labels to each frames')

def assign_label(df,root,filenames):
  for filename in filenames:
    #processed files contain '_' hence we can omit those
    if '_' in filename:
      continue
    #get the extension from the file
    ext = filename.split('.')[-1]
    split_array = filename.split('.')[0].split('-')
    #extract id and frame
    file_id,frame_id = '-'.join(split_array[:2]), split_array[-1]
    #adl files doesnt contain falls hence assigning label 0 to all files
    if 'adl' in file_id:
      label = 0
    else:
      #check the label in dataframe for id and frame
      df_label = int(df[(df[0] == file_id) & (df[1] == int(frame_id))][2])
      label = 1 if df_label == 1 else 0
    #Rename the files
    name = f'{file_id}_{frame_id}_{label}.{ext}'
    new_path = os.path.join(root, name)
    old_path = os.path.join(root,filename)
    os.rename(old_path,new_path)
    # create a entry in the dataframe for new path of the frame file
    df.loc[(df[0] == file_id) & (df[1] == int(frame_id)), 'path'] = new_path
    df.loc[(df[0] == file_id) & (df[1] == int(frame_id)), 2] = label


def enrich_labels(df,dataPath,folder=True):
  '''
  Embed labels into the data filenames
  df: data frame consisting of details on labels
  datapath: the path of the folder containing frames
  folder: indicates the structure of the data
  '''
  if folder:
    root, dirs, _ = next(os.walk(dataPath))
    dir_array = [os.path.join(root,dir) for dir in dirs]
    for path in dir_array:
      root, _, files = next(os.walk(path))
      assign_label(df,root,files)
  else:
    root, _, files = next(os.walk(dataPath))
    assign_label(df, root, files)




INFO:withoutlevel:======================================================================================================================================================================================================== 

STEP 2: Assign labels to each frames


In [ ]:
# create dataframe for further processing using the CSV file data
csv = root + '/dataset/urfd/csv'
df_fall = pd.read_csv('/content/dataset/urfd/csv/urfall-cam0-falls.csv',header=None)
df_adl = pd.read_csv('/content/dataset/urfd/csv/urfall-cam0-adls.csv',header=None)
dfs = pd.concat([df_fall,df_adl],ignore_index=True)
dfs = dfs[[0,1,2]]
dfs['path'] = None
logger_t.info(f"Enrich labels into the filenames extractend from the CSV files")
t = t_set()
enrich_labels(dfs,'/content/dataset/urfd/frame')
logger.info(f'Elapsed time for modifying filenames: {t_diff(t)}s.')
wandb.summary["ModifyFileNames_totaltime"] = w_diff(t)

INFO:withoutlevel:Enrich labels into the filenames extractend from the CSV files
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:withlevel:Elapsed time for modifying filenames: 0:00:56.736262s.


## Create Subset

In [ ]:

import numpy as np
import os
import shutil
from tqdm.notebook import tqdm
import torch
import random
from torch.utils.data import random_split


def safedirs(path):
  if not os.path.exists(path):
    os.makedirs(path)

def createDataset(df,dataPath,destPath,ids,clip_len=50,size=60):
  random.seed(cfg.seed_value)
  logger.info(f'Seed value is {cfg.seed_value}')
  assert size % 60 == 0, "Size should be in the multiple of 60"
  assert type(ids) == list, "expecting a list of strings"
  data_files = {i:[] for i in ids}
  for root,dir,files in os.walk(dataPath):
    for f in files:
      id = f.split('_')[0]
      if id in ids:
        data_files[id].append(os.path.join(root,f))
  for subset in range(size//60):
    t = t_set()
    for idx in range(len(list(data_files.keys()))):
      path = sorted(data_files[list(data_files.keys())[idx]])
      topic = (list(data_files.keys())[idx])
      while True:
        root_path = os.path.join(destPath, topic.split('-')[0], topic+'_'+str(subset))
        safedirs(root_path)
        rand_n = random.randint(0,len(path)-clip_len)
        frame_path = path[rand_n : rand_n+clip_len]
        labels = []
        for fp in frame_path:
          shutil.copy(fp, os.path.join(root_path, os.path.basename(fp)))
          #frame is the datapath with label at the end of the filename
          labels.append(int(fp.split('.')[0][-1]))
        if topic.split('-')[0] == 'adl':
          break
        elif topic.split('-')[0] == 'fall' and labels.count(1) > 4:
          break
        shutil.rmtree(root_path)
    logger.info(f'time elapsed for each subset {subset} is {t_diff(t)}')



In [ ]:
from torch.utils.data import random_split
#120 image dataset
#shutil.rmtree('/content/subset')
logger_t.info(f'{"="*100} \n\nSTEP 3: Create subset of 60/120/180 files of 50 Frames each from the Dataset')

logger.info(f'Size of the subset: {cfg.data_size}')
ids = dfs[0].unique()[:60]
train_size = int(cfg.split*len(ids))
val_size = int(cfg.split*(len(ids) - train_size))
test_size = len(ids) - (train_size + val_size)
train_ids,valid_ids,test_ids = random_split(ids,[train_size,val_size,test_size],generator=torch.Generator().manual_seed(1))
logger_t.info(f'train_size={train_size}\nvalidation_size={val_size}\ntest_size={test_size}')
wandb.summary['trainset_size'] = train_size
wandb.summary['validationset_size'] = val_size
wandb.summary['testset_size'] = test_size
logger_t.info(f'train_ids={train_ids}\nvalidation ids={valid_ids}\ntest_ids={test_ids}')

train_destPath = os.path.join(os.getcwd(),'subset/train')
valid_destPath = os.path.join(os.getcwd(),'subset/valid')
test_destPath = os.path.join(os.getcwd(),'subset/test')
logger_t.info(f'train path= {train_destPath}\nvalidation path= {valid_destPath}\nTest path= {test_destPath}')

t = t_set()
createDataset(dfs,'/content/dataset/urfd/frame', train_destPath, ids = list(train_ids), clip_len=cfg.frameCount, size=cfg.data_size)
logger.info(f"Time elapsed for creating Train subset is {t_diff(t)}")
wandb.summary['trainSubset_creationTime'] = w_diff(t)
t = t_set()
createDataset(dfs,'/content/dataset/urfd/frame', valid_destPath, ids = list(valid_ids), clip_len=cfg.frameCount, size=cfg.data_size)
logger.info(f"Time elapsed for creating Validation subset is {t_diff(t)}")
wandb.summary['valSubset_creationTime'] = w_diff(t)
t = t_set()
#createDataset(dfs,'/content/dataset/urfd/frame',test_destPath,ids = list(test_ids),size=size)
createDataset(dfs,'/content/dataset/urfd/frame', test_destPath, ids = list(ids), clip_len=cfg.frameCount, size=cfg.data_size)
logger.info(f"Time elapsed for creating Test subset is {t_diff(t)}")
wandb.summary['testSubset_creationTime'] = w_diff(t)


INFO:withoutlevel:==================================================================================================== 

STEP 3: Create subset of 60/120/180 files of 50 Frames each from the Dataset
INFO:withlevel:Size of the subset: 120
INFO:withoutlevel:train_size=48
validation_size=9
test_size=3
INFO:withoutlevel:train_ids=<torch.utils.data.dataset.Subset object at 0x7f7c810d1090>
validation ids=<torch.utils.data.dataset.Subset object at 0x7f7c810d1850>
test_ids=<torch.utils.data.dataset.Subset object at 0x7f7c810d1690>
INFO:withoutlevel:train path= /content/subset/train
validation path= /content/subset/valid
Test path= /content/subset/test
INFO:withlevel:Seed value is 43
INFO:withlevel:time elapsed for each subset 0 is 0:00:09.349669
INFO:withlevel:time elapsed for each subset 1 is 0:00:08.427887
INFO:withlevel:Time elapsed for creating Train subset is 0:00:17.829298
INFO:withlevel:Seed value is 43
INFO:withlevel:time elapsed for each subset 0 is 0:00:01.692962
INFO:withlevel:time e

## Embedding data

In [ ]:
from collections import defaultdict
from skimage.metrics import structural_similarity
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time,os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing


def person_detector(imageMatrix,label,mode,folder):
  '''
  Moving object detection by frame differencing
  input: torch tensor object of shape [batch,50,3,480,640] and labels array
  output: tuple of torch tensor [batch,1,224,224] and label object 
  '''
  info = ['adl','fall']
  bbox_path = os.path.join(os.getcwd(), 'boundingBox', mode, info[label], folder)
  logger_t.info(f'Working on creating ... {bbox_path} set')
  safedirs(bbox_path)
  temp_img = imageMatrix
  center_points = []
  for i in range(0,temp_img.shape[0] - 2):
    img1 = cv2.medianBlur(cv2.cvtColor(temp_img[i],cv2.COLOR_BGR2GRAY),5)
    img2 = cv2.medianBlur(cv2.cvtColor(temp_img[i+2],cv2.COLOR_BGR2GRAY),5)
    (score,diff) = structural_similarity(img1,img2,full=True)
    diff = (diff * 255).astype('uint8')
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    dilate_frame = cv2.dilate(thresh, None, iterations=5)
    contours = cv2.findContours(dilate_frame.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1] 
    val = np.argmax([cv2.contourArea(c) for c in contours])
    x,y,w,h = cv2.boundingRect(contours[val])
    cv2.rectangle(img1, (x, y), (x + w, y + h), (36,255,12), 2)
    path = os.path.join(bbox_path, f'image_{i}.jpg')
    cv2.imwrite(path, img1)
    center = (int((x+w)/2), int((y+h)/2))
    center_points.append(center)
  return center_points

def get_transform():
  transform = tf.keras.Sequential([
                                   preprocessing.Resizing(224,224),
                                   preprocessing.Normalization()
  ])
  return transform


def image_embedding(images,label,mode,filename):
  transform = get_transform()
  info = ['adl', 'fall']
  trajectory_path = os.path.join(os.getcwd(), 'trajectories', mode, info[label])
  safedirs(trajectory_path)
  logger_t.info(f'creating trajectory image at {trajectory_path}')
  bkg = np.zeros((480,640),np.uint8)
  t=t_set()
  points = person_detector(images,label,mode,filename) 
  logger.info(f'Elapsed time for Bounding box calculation is {t_diff(t)}')
  wandb.log({'boundingBox':{'time':w_diff(t)}})
  for p in range(len(points)-1):
    bkg = cv2.line(bkg,points[p],points[p+1],(255,0,0),5)
  bkg = transform(tf.expand_dims(bkg,2))
  path = os.path.join(trajectory_path, filename+'.jpg')
  cv2.imwrite(path, bkg.numpy())


def embedded_data(datapath):
  data_dict = defaultdict(list)
  for root,dir,files in os.walk(datapath):
      for f in files:
        frame_path = os.path.join(root,f)
        #print(root)
        folder = os.path.basename(root)
        data_dict[folder].append(frame_path)
  for key in list(data_dict.keys()):
    frames, labels = [], []
    for f in sorted(data_dict[key]):
      labels.append(int(f.split('.')[0][-1]))
      temp = f.split('/')
      mode = temp[3]
      filename = temp[5]
      img = cv2.cvtColor(cv2.imread(f),cv2.COLOR_BGR2RGB)
      frames.append(img)
    frames = np.stack(frames,axis=0)
    t=t_set()
    image_embedding(frames, np.max(labels), mode, filename)
    logger.info(f"Total time taken for {key} is {t_diff(t)}")
    wandb.log({'trajectory':{'time':w_diff(t)}})


logger_t.info(f'{"="*100} \n\nSTEP 4: Create Trajectory images\n')
t = t_set()
root = os.getcwd()
embedded_data(root + '/subset/train')
embedded_data(root + '/subset/valid')
embedded_data(root + '/subset/test')
logger.info(f'Total time taken to create trajectory images: {t_diff(t)}')
wandb.summary["Trajectory_totaltime"] = w_diff(t)


INFO:withoutlevel:==================================================================================================== 

STEP 4: Create Trajectory images

INFO:withoutlevel:creating trajectory image at /content/trajectories/train/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/adl/adl-05_0 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.649385
INFO:withlevel:Total time taken for adl-05_0 is 0:00:02.965644
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/adl/adl-07_0 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.565487
INFO:withlevel:Total time taken for adl-07_0 is 0:00:02.600258
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/adl/adl-04_1 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.611974

## Artifact

In [ ]:
bbox = wandb.Artifact('boundingBox', type='dataset')
bbox.add_dir('boundingBox')
wandb.log_artifact(bbox)

wandb: Adding directory to artifact (./boundingBox)... Done. 13.5s


In [ ]:
trajectory = wandb.Artifact('trajectory', type='dataset')
trajectory.add_dir('trajectories')
wandb.log_artifact(trajectory)

wandb: Adding directory to artifact (./trajectories)... Done. 0.5s


## Data Generators

In [ ]:
logger_t.info(f'{"="*100} \n\nSTEP 5: Create Data generators\n')
datagen = tf.keras.preprocessing.image.ImageDataGenerator()
logger.info(f'batch size: {cfg.batch_size}')
root = os.getcwd()
train = datagen.flow_from_directory(root + '/trajectories/train',target_size=(224, 224),color_mode='grayscale',batch_size=cfg.batch_size)
valid = datagen.flow_from_directory(root + '/trajectories/valid',target_size=(224, 224),color_mode='grayscale',batch_size=cfg.batch_size)
test = datagen.flow_from_directory(root + '/trajectories/test',target_size=(224, 224),color_mode='grayscale',batch_size=cfg.batch_size)

INFO:withoutlevel:==================================================================================================== 

STEP 5: Create Data generators

INFO:withlevel:batch size: 32


Found 96 images belonging to 2 classes.
Found 18 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


## Model

In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,GlobalAveragePooling2D,Dense,Dropout
from tensorflow.keras.metrics import Precision, Recall
from wandb.keras import WandbCallback
logger_t.info(f'{"="*100} \n\nSTEP 5: Create Model and train the model\n')



model = tf.keras.Sequential([
              tf.keras.Input(shape=(224,224,1)),
              Conv2D(32,3,strides=2,padding='same',activation=cfg.activation,use_bias=False),
              MaxPool2D(),
              Conv2D(64,3,strides=2,padding='same',activation=cfg.activation,use_bias=False),
              MaxPool2D(),
              Conv2D(128,3,strides=2,padding='same',activation=cfg.activation,use_bias=False),
              MaxPool2D(),
              GlobalAveragePooling2D(),
              Dense(128),
              Dropout(cfg.dropout_rate),
              Dense(64),
              Dropout(cfg.dropout_rate),
              Dense(2,activation='sigmoid')])


model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', Precision(), Recall()])
logger.info(f'{model.summary()}')



INFO:withoutlevel:==================================================================================================== 

STEP 5: Create Model and train the model

INFO:withlevel:None


Model: "sequential_234"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 112, 112, 32)      288       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18432     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73728     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 128)            

## Training

In [ ]:
# fit model
history = model.fit_generator(generator=train,
                              validation_data=valid,
                              epochs=cfg.epochs,
                              verbose=1,
                              callbacks = [WandbCallback()])


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
3/3 [==============================] - 4s 599ms/step - loss: 2.3626 - accuracy: 0.5729 - precision: 0.5612 - recall: 0.5729 - val_loss: 0.4090 - val_accuracy: 0.9444 - val_precision: 0.7826 - val_recall: 1.0000
Epoch 2/15
3/3 [==============================] - 1s 376ms/step - loss: 1.1731 - accuracy: 0.6667 - precision: 0.5982 - recall: 0.6979 - val_loss: 0.3090 - val_accuracy: 0.8889 - val_precision: 0.9333 - val_recall: 0.7778
Epoch 3/15
3/3 [==============================] - 1s 370ms/step - loss: 0.6305 - accuracy: 0.8542 - precision: 0.8125 - recall: 0.6771 - val_loss: 0.5805 - val_accuracy: 0.7222 - val_precision: 0.7500 - val_recall: 0.6667
Epoch 4/15
3/3 [==============================] - 1s 362ms/step - loss: 0.5083 - accuracy: 0.8646 - precision: 0.7978 - recall: 0.7396 - val_loss: 0.3383 - val_accuracy: 0.8889 - val_precision: 0.8000 - val_recall: 0.8889
Epoch 5/15
3/3 [==============================] - 1s 375ms/step - loss: 0.4470 - accuracy: 0.8958 - precision: 0

In [ ]:
best_model = wandb.restore("model-best.h5")
model.load_weights(best_model.name)

In [ ]:
results = model.evaluate(test)

4/4 [==============================] - 0s 95ms/step - loss: 0.0979 - accuracy: 0.9583 - precision: 0.9576 - recall: 0.9417


In [ ]:
wandb.summary['test_loss'] = results[0]
wandb.summary['test_accuracy'] = results[1]
wandb.summary['test_precision'] = results[2]
wandb.summary['test_recall'] = results[3]

In [ ]:
wandb.finish()

_runtime,1285
_timestamp,1627567046
_step,554
Download_TotalTime,404.57377
ModifyFileNames_totaltime,56.73974
trainset_size,48
validationset_size,9
testset_size,3
trainSubset_creationTime,17.83273
valSubset_creationTime,3.20152
testSubset_creationTime,27.45594


_runtime,▁▁▂▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇████
_timestamp,▁▁▂▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁
accuracy,▁▃▆▆▆▆▇▇▇██████
precision,▁▂▅▅▄▅▇▇▇▇█████
recall,▁▃▃▄▇▆▇▆▇▇▇████
val_loss,▆▅█▅▂▃▅▆▃▂▁▁▁▂▂
val_accuracy,▇▅▁▅▇▇▄▂▇▇███▇▇
val_precision,▂▆▁▂▆▆▅▅▆▆███▆▆
